# INPUT

In [1]:
grid = []
with open('input.txt') as f:
    for line in f:
        grid.append(list(line.strip()))


# PART 1

In [2]:
def check(grid, x, y, dim_x, dim_y):
    counter = 0
    for x_loc in range(max(0, x-1), min(dim_x - 1, x + 1)+1):
        for y_loc in range(max(0, y-1), min(dim_y - 1, y + 1)+1):
            if grid[x_loc][y_loc] == '@':
                counter+=1
                if counter == 5:
                    return 0
    return 1 

In [3]:
dim_x = len(grid[0])
dim_y = len(grid)

forkable = 0
for x in range(0, dim_x):
    for y in range(0, dim_y):
        if grid[x][y] == '@':
                forkable += check(grid, x, y, dim_x, dim_y)


forkable


1626

# PART 2

In [4]:
dim_x = len(grid[0])
dim_y = len(grid)

forkable = 0
forked_iteration = 0
iter_one = True
new_grid = grid
while forked_iteration > 0 or iter_one:
    iter_one = False
    forked_iteration = 0
    grid = new_grid
    for x in range(0, dim_x):
        for y in range(0, dim_y):
            if grid[x][y] == '@':
                can_be_removed = check(grid, x, y, dim_x, dim_y)
                forked_iteration += can_be_removed
                if can_be_removed == 1:
                    new_grid[x][y] = '.'
                else:
                    new_grid[x][y] = grid[x][y]

    forkable +=forked_iteration


forkable

9173

# BONUS

In [5]:
import torch
import torch.nn.functional as F

input_matrix = []
with open('input.txt', 'r') as file:
    for line in file:
        input_matrix.append([1 if x =='.' else -1 for x in line.strip()])

# PART 1

In [6]:
input_tensor = torch.tensor(input_matrix, dtype=torch.float32)
offset = 10 # greater that 4 to be able to handle counting of surrounding values
device = 'cuda' if torch.cuda.is_available() else 'cpu'
input_tensor_padded = F.pad(input_tensor, (1, 1, 1, 1), mode='constant', value=1).unsqueeze(0).unsqueeze(0).to(device)
kernel = torch.tensor([
    [[
        [1, 1, 1],
        [1, -offset, 1],
        [1, 1, 1]
    ]]
], dtype=torch.float32).to(device)

In [7]:
y = (F.conv2d(input_tensor_padded, kernel).squeeze()) - offset
mask = torch.sign(y).relu()
count = torch.sum(mask).item()

int(count)


1626

# PART 2

In [8]:
count = 1
total = 0
while count > 0:
    y = (F.conv2d(input_tensor_padded, kernel, padding=0).squeeze()) - offset
    mask = torch.sign(y).relu()
    count = torch.sum(mask).item()
    mask = F.pad(mask, (1, 1, 1, 1), mode='constant', value=0).unsqueeze(0).unsqueeze(0)*2

    input_tensor_padded += mask

    total += count

int(total)

9173